In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
# 定义一个简单的线性模型 y = wx + b
model = nn.Linear(in_features=1, out_features=1)

# 打印模型参数（w 和 b）
for name, param in model.named_parameters():
    print(name, param.data)


weight tensor([[-0.8461]])
bias tensor([0.4212])


In [9]:
x = torch.tensor([[1.0],[2.0],[3.0],[4.0]])
y = torch.tensor([[5.0],[7.0],[9.0],[11.0]]) 

model = nn.Linear(1,1)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(),lr=0.01)

for i in range(1000):
    y_pre = model(x)
    loss = loss_fn(y_pre,y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

for name,param in model.named_parameters():
    print(name,param.item())


weight 1.9831494092941284
bias 3.048121213912964


In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
epoches = 5
# ======================
# 1. 数据准备
# ======================
transform = transforms.ToTensor()

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)


class MLP(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.layers = nn.Sequential(

            nn.Flatten(),
            nn.Linear(28*28,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10),
        )
    def forward(self,x):
        return self.layers(x)
         


model = MLP()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

for epoch in range(epoches):
    for X,y in train_loader:
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
correct = 0
total = 0
with torch.no_grad():
    for X, y in test_loader:
        y_pred = model(X)
        predicted = torch.argmax(y_pred, dim=1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

print(f"测试集准确率: {100 * correct / total:.2f}%")

测试集准确率: 97.94%


In [16]:
epoches = 5
# ======================
# 1. 数据准备
# ======================
transform = transforms.ToTensor()

train_dataset = torchvision.datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root="./data", train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=1000, shuffle=False)

class CNN(nn.Module):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.conv_layer = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2),
            nn.Conv2d(32,64,kernel_size=3,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2,2)
        )
        self.fn_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 7 * 7, 128),
            nn.ReLU(),
            nn.Linear(128,10)
        )
    def forward(self,x):
        x = self.conv_layer(x)
        x =self.fn_layer(x)
        return x
    
model = CNN()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=0.001)

for epoch in range(epoches):
    for X,y in train_loader:
        y_pred = model(X)
        loss = loss_fn(y_pred,y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

correct = 0
predict = 0

with torch.no_grad():
    for X,y in test_loader:
        y_pre = model(X)
        predicted = torch.argmax(y_pre,dim=1)
        predict += y.size(0)
        correct += (predicted == y).sum().item()
print(f"正确率是{100 * correct/predict:.2f}")

正确率是99.05


In [28]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# 1. 读取 CSV
data = pd.read_csv("./Fer-2013/fer2013.csv")

# 2. 定义 Dataset
class FER2013Dataset(Dataset):
    def __init__(self, dataframe, usage, transform=None):
        self.data = dataframe[dataframe['Usage'] == usage]
        self.transform = transform

        self.emotions = self.data['emotion'].values
        self.pixels = self.data['pixels'].values

    def __len__(self):
        return len(self.emotions)

    def __getitem__(self, idx):
        # emotion 标签
        emotion = int(self.emotions[idx])

        # pixels 转换成 numpy 数组并 reshape 成 48x48
        pixels = np.array(self.pixels[idx].split(), dtype=np.uint8)
        image = pixels.reshape(48, 48)  # 灰度图

        # 转成 PIL Image 或 tensor
        if self.transform:
            image = self.transform(image)

        return image, emotion

from torchvision import transforms

# 训练集增强
train_transform = transforms.Compose([
    transforms.ToPILImage(),                # numpy -> PIL
    transforms.RandomHorizontalFlip(p=0.5), # 50% 概率水平翻转
    transforms.RandomRotation(15),          # 随机旋转 ±15°
    transforms.RandomResizedCrop(48, scale=(0.9, 1.1)), # 随机缩放裁剪
    transforms.ColorJitter(brightness=0.2, contrast=0.2), # 光照扰动
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))    # 灰度图只需一个均值/方差
])

# 验证/测试集不需要随机增强，只做标准化
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


# 4. 创建训练集 & 测试集
train_dataset = FER2013Dataset(data, usage="Training", transform=transform)
val_dataset = FER2013Dataset(data, usage="PublicTest", transform=transform)
test_dataset = FER2013Dataset(data, usage="PrivateTest", transform=transform)

# 5. DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print("训练集大小:", len(train_dataset))
print("验证集大小:", len(val_dataset))
print("测试集大小:", len(test_dataset))


训练集大小: 28709
验证集大小: 3589
测试集大小: 3589


In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epoches = 5
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1), 
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 48 -> 24

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2),  # 24 -> 12

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)   # 12 -> 6
        )

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128 * 6 * 6, 256),
            nn.ReLU(),
            nn.Dropout(0.5),        # 加 dropout 防止过拟合
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 7)       # FER-2013 七类
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.fc_layers(x)
        return x
    
num_epochs = 30   # FER-2013 建议多训练一些
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CNN().to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)  
# 每10个epoch学习率减半

for epoch in range(num_epochs):
    # ========= 训练阶段 =========
    model.train()
    train_loss, correct, total = 0, 0, 0

    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        # 前向传播
        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        # 反向传播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # 统计训练指标
        train_loss += loss.item() * X.size(0)
        _, predicted = torch.max(y_pred, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()

    train_acc = 100 * correct / total
    train_loss /= total

    # ========= 验证阶段 =========
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for X, y in val_loader:   # 注意要准备 val_loader
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss = loss_fn(y_pred, y)

            val_loss += loss.item() * X.size(0)
            _, predicted = torch.max(y_pred, 1)
            total += y.size(0)
            correct += (predicted == y).sum().item()

    val_acc = 100 * correct / total
    val_loss /= total

    # 调整学习率
    scheduler.step()

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% "
          f"| Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}%")


Epoch [1/30] Train Loss: 1.6690 | Train Acc: 32.85% | Val Loss: 1.4721 | Val Acc: 43.47%
Epoch [2/30] Train Loss: 1.4365 | Train Acc: 44.44% | Val Loss: 1.3554 | Val Acc: 47.56%
Epoch [3/30] Train Loss: 1.3296 | Train Acc: 48.96% | Val Loss: 1.2947 | Val Acc: 49.60%
Epoch [4/30] Train Loss: 1.2556 | Train Acc: 52.08% | Val Loss: 1.2550 | Val Acc: 51.60%
Epoch [5/30] Train Loss: 1.1916 | Train Acc: 54.54% | Val Loss: 1.2288 | Val Acc: 52.30%
Epoch [6/30] Train Loss: 1.1371 | Train Acc: 56.60% | Val Loss: 1.2372 | Val Acc: 52.05%
Epoch [7/30] Train Loss: 1.0930 | Train Acc: 58.36% | Val Loss: 1.2219 | Val Acc: 52.72%
Epoch [8/30] Train Loss: 1.0413 | Train Acc: 60.52% | Val Loss: 1.2269 | Val Acc: 54.25%
Epoch [9/30] Train Loss: 1.0037 | Train Acc: 62.00% | Val Loss: 1.2064 | Val Acc: 54.64%
Epoch [10/30] Train Loss: 0.9509 | Train Acc: 63.93% | Val Loss: 1.2528 | Val Acc: 53.83%
Epoch [11/30] Train Loss: 0.8547 | Train Acc: 67.55% | Val Loss: 1.2623 | Val Acc: 54.64%
Epoch [12/30] Train

KeyboardInterrupt: 